In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet169,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexByteplot"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet169(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1664)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1664)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.9min


[CV] ........................... C=0.01, score=0.895669, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.891176, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.883858, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.892644, total= 3.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.898910, total= 3.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.889442, total= 3.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.888889, total= 3.5min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  5.3min


[CV] ............................ C=0.1, score=0.888780, total= 5.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.881944, total= 5.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.880859, total= 5.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.878937, total= 5.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.888235, total= 5.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.873529, total= 5.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.877691, total= 5.8min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] .................... C=100000000.0, score=0.833665, total= 3.6min
[CV] ................... C=1000000000.0, score=0.869141, total= 3.6min
[CV] ................... C=1000000000.0, score=0.866667, total= 3.5min
[CV] ................... C=1000000000.0, score=0.862103, total= 3.3min
[CV] ................... C=1000000000.0, score=0.887240, total= 3.7min
[CV] ................... C=1000000000.0, score=0.867195, total= 3.6min
[CV] ................... C=1000000000.0, score=0.865157, total= 3.7min
[CV] ................... C=1000000000.0, score=0.870775, total= 3.5min
[CV] ................... C=1000000000.0, score=0.862550, total= 3.5min
[CV] .................. C=10000000000.0, score=0.879883, total= 3.2min
[CV] .................. C=10000000000.0, score=0.859100, total= 3.0min
[CV] .................. C=10000000000.0, score=0.867647, total= 3.0min
[CV] .................. C=10000000000.0, score=0.848425, total= 3.0min
[CV] .................. C=10000000000.0, score=0.888229, total= 2.9min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 21.2min remaining:   50.8s


[CV] .................. C=10000000000.0, score=0.859252, total= 3.1min
[CV] .................. C=10000000000.0, score=0.854167, total= 2.7min
[CV] .................. C=10000000000.0, score=0.877106, total= 3.0min
[CV] .................. C=10000000000.0, score=0.880716, total= 2.6min
[CV] .................. C=10000000000.0, score=0.869522, total= 2.6min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 21.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([194.58392618, 336.28169677, 280.1392669 , 228.57394974,
        217.91043212, 220.15233579, 219.31231174, 218.26064911,
        217.44374077, 220.70916958, 216.49067123, 210.42999878,
        174.27233341]),
 'mean_score_time': array([0.05686252, 0.04004583, 0.03217642, 0.03043749, 0.02657282,
        0.03033688, 0.02870345, 0.02836421, 0.02836857, 0.02929373,
        0.0252001 , 0.02077277, 0.01405663]),
 'mean_test_score': array([0.89206788, 0.88723362, 0.87352013, 0.86730466, 0.86829124,
        0.87204025, 0.8683899 , 0.86986977, 0.86592344, 0.86740331,
        0.86661405, 0.86592344, 0.8683899 ]),
 'mean_train_score': array([0.95029855, 0.98056439, 0.98730696, 0.98153073, 0.98173872,
        0.98443217, 0.98263871, 0.98263741, 0.98029254, 0.98577142,
        0.9776129 , 0.98076504, 0.98366852]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.89207, std: 0.00689, params: {'C': 0.01},
 mean: 0.88723, std: 0.00723, params: {'C': 0.1},
 mean: 0.87352, std: 0.00860, params: {'C': 1.0},
 mean: 0.86730, std: 0.00724, params: {'C': 10.0},
 mean: 0.86829, std: 0.00664, params: {'C': 100.0},
 mean: 0.87204, std: 0.00888, params: {'C': 1000.0},
 mean: 0.86839, std: 0.00726, params: {'C': 10000.0},
 mean: 0.86987, std: 0.00631, params: {'C': 100000.0},
 mean: 0.86592, std: 0.00980, params: {'C': 1000000.0},
 mean: 0.86740, std: 0.00917, params: {'C': 10000000.0},
 mean: 0.86661, std: 0.01228, params: {'C': 100000000.0},
 mean: 0.86592, std: 0.00888, params: {'C': 1000000000.0},
 mean: 0.86839, std: 0.01235, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.8921
